In [16]:
from pyhive import hive
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, when
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id

In [17]:
spark = SparkSession.builder.appName('MIGRACAO_HIVE').getOrCreate()

In [18]:
df_total_votos_candidato = spark \
    .read \
    .parquet('../data/processed/total_votos_candidato.parquet')
df_total_votos_candidato.show()

+-----------+--------+------------+------------+--------------------+--------------------+----------------+----------------+----------------------------+------------------+-----------------------+
|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|CD_SIT_TOT_TURNO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|
+-----------+--------+------------+------------+--------------------+--------------------+----------------+----------------+----------------------------+------------------+-----------------------+
|       2018|       6|       61018|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|               2|   ELEITO POR QP|                        2179|             16124|                  13.51|
|       2018|       6|       61018|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|               5|        SUPLENTE|                           0|             16124|                    0.0|
|       2018|  

In [19]:
df_total_votos_candidato.count()

4382775

In [20]:
df_total_votos_candidato = df_total_votos_candidato \
    .select(
        'NR_CANDIDATO',
        'NM_CANDIDATO',
        'NM_URNA_CANDIDATO',
        'DS_SIT_TOT_TURNO',
        'TOTAL_VOTOS_CANDIDATO_CIDADE',
        'TOTAL_VOTOS_CIDADE',
        'PERCENTUAL_VOTOS_CIDADE',
        'ANO_ELEICAO',
        'CD_CARGO',
        'CD_MUNICIPIO',
        'CD_SIT_TOT_TURNO'

    )

df_total_votos_candidato.show()

+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+
|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|CD_SIT_TOT_TURNO|
+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+
|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|   ELEITO POR QP|                        2179|             16124|                  13.51|       2018|       6|       61018|               2|
|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|        SUPLENTE|                           0|             16124|                    0.0|       2018|       6|       61018|               5|
|        1771|J

In [21]:
lista_colunas = df_total_votos_candidato.columns

In [22]:
lista_colunas

['NR_CANDIDATO',
 'NM_CANDIDATO',
 'NM_URNA_CANDIDATO',
 'DS_SIT_TOT_TURNO',
 'TOTAL_VOTOS_CANDIDATO_CIDADE',
 'TOTAL_VOTOS_CIDADE',
 'PERCENTUAL_VOTOS_CIDADE',
 'ANO_ELEICAO',
 'CD_CARGO',
 'CD_MUNICIPIO',
 'CD_SIT_TOT_TURNO']

In [23]:
for  coluna in lista_colunas:
    df_total_votos_candidato.groupBy(coluna).count().orderBy('count', ascending=False).show()

+------------+-----+
|NR_CANDIDATO|count|
+------------+-----+
|        1084| 1290|
|       51022| 1290|
|        1522| 1290|
|        1580| 1290|
|        1990| 1290|
|       65300| 1290|
|       15100| 1290|
|       22223| 1290|
|       22555| 1290|
|       18800| 1290|
|        1270| 1290|
|        2866| 1290|
|        1322| 1290|
|        1025| 1290|
|        3000| 1290|
|        2811| 1290|
|        1903| 1290|
|       19200| 1290|
|        5173| 1290|
|       51123| 1290|
+------------+-----+
only showing top 20 rows



+--------------------+-----+
|        NM_CANDIDATO|count|
+--------------------+-----+
|LUIZ CARLOS DE PAULA| 1935|
|MARIA LUCIA DA SILVA| 1935|
| MARIA JOSE DA SILVA| 1935|
|MARLON FARIAS DA LUZ| 1290|
|LENIR MARIA DE OL...| 1290|
|CLEOMENES ANTUNES...| 1290|
|     PAULO NISHIKAWA| 1290|
|LUIZ FERNANDO TEI...| 1290|
|RICARDO HOLZ VIDA...| 1290|
|ALENCAR SANTANA B...| 1290|
|FREDERICO BRAUN D...| 1290|
|VITOR ALEXANDRE R...| 1290|
|IVONE FERNANDES D...| 1290|
| THIAGO FELIPE NEVES| 1290|
|ARNALDO CALIL PER...| 1290|
|CÍCERO JOSÉ DE GO...| 1290|
|EDUARDO LIMA DE S...| 1290|
|JEFFERSON ALVES D...| 1290|
|JORGE WILSON GONÇ...| 1290|
|SEBASTIÃO DOS SAN...| 1290|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|   NM_URNA_CANDIDATO|count|
+--------------------+-----+
|  PROFESSOR FERNANDO| 2580|
|            SERGINHO| 1935|
|       RICARDO SILVA| 1935|
|          NEIA COSTA| 1935|
|     FERNANDO PETITI| 1290|
|       MARCUS DANTAS| 1290|
|ENFERMEIRA EDITH

In [24]:
list_cidade = [ linha['CD_MUNICIPIO'] for linha in df_total_votos_candidato.select('CD_MUNICIPIO').distinct().collect() ]
list_cidade

[71510,
 68098,
 61344,
 66613,
 67857,
 64874,
 61859,
 65013,
 71374,
 61786,
 63835,
 61654,
 64459,
 67911,
 65196,
 69515,
 65110,
 61158,
 61328,
 64033,
 66338,
 67474,
 65714,
 64254,
 61212,
 62456,
 61514,
 61077,
 62073,
 70912,
 71110,
 61298,
 71978,
 61271,
 62774,
 70718,
 61905,
 63037,
 68497,
 66257,
 67334,
 67156,
 69736,
 71650,
 67997,
 64610,
 62375,
 64335,
 66494,
 70394,
 71099,
 71218,
 63096,
 67490,
 67431,
 63673,
 63690,
 63711,
 70939,
 70190,
 63193,
 68411,
 63452,
 63797,
 66095,
 65072,
 70114,
 68470,
 71250,
 63339,
 70971,
 70556,
 66915,
 61417,
 62421,
 72451,
 71390,
 69159,
 64971,
 68594,
 64297,
 64998,
 67393,
 68217,
 63770,
 61778,
 63479,
 63258,
 68373,
 65579,
 61565,
 68950,
 61824,
 70599,
 64394,
 61131,
 64114,
 68535,
 64513,
 64530,
 63398,
 61697,
 69310,
 62790,
 63614,
 61662,
 65790,
 68659,
 64190,
 66753,
 64572,
 61140,
 67415,
 61093,
 70998,
 63371,
 62448,
 71838,
 66850,
 62596,
 64173,
 65455,
 65773,
 70955,
 69299,


In [25]:
df_total_votos_candidato = df_total_votos_candidato.withColumn('ID_LINHA', monotonically_increasing_id()) 

In [26]:
df_total_votos_candidato.show()

+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+
|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|CD_SIT_TOT_TURNO|ID_LINHA|
+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+
|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|   ELEITO POR QP|                        2179|             16124|                  13.51|       2018|       6|       61018|               2|       0|
|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|        SUPLENTE|                           0|             16124|                    0.0|       2018|       6|       610

In [27]:
numero_linha_particao = 500
df_total_votos_candidato = df_total_votos_candidato.withColumn('PARTICAO_ID', (df_total_votos_candidato['ID_LINHA'] / numero_linha_particao).cast('integer'))
df_total_votos_candidato.show()

+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+-----------+
|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|CD_SIT_TOT_TURNO|ID_LINHA|PARTICAO_ID|
+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+-----------+
|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|   ELEITO POR QP|                        2179|             16124|                  13.51|       2018|       6|       61018|               2|       0|          0|
|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|        SUPLENTE|                           0|             16124|       

In [46]:
particoes = df_total_votos_candidato.select('PARTICAO_ID').distinct().collect()

In [47]:
particoes.sort(reverse=True)

In [50]:
len(particoes)

8773

In [53]:
for chave , particao in enumerate(particoes):
    print(chave, particao)

0 Row(PARTICAO_ID=120260312)
1 Row(PARTICAO_ID=120260311)
2 Row(PARTICAO_ID=120260310)
3 Row(PARTICAO_ID=120260309)
4 Row(PARTICAO_ID=120260308)
5 Row(PARTICAO_ID=120260307)
6 Row(PARTICAO_ID=120260306)
7 Row(PARTICAO_ID=120260305)
8 Row(PARTICAO_ID=120260304)
9 Row(PARTICAO_ID=120260303)
10 Row(PARTICAO_ID=120260302)
11 Row(PARTICAO_ID=120260301)
12 Row(PARTICAO_ID=120260300)
13 Row(PARTICAO_ID=120260299)
14 Row(PARTICAO_ID=120260298)
15 Row(PARTICAO_ID=120260297)
16 Row(PARTICAO_ID=120260296)
17 Row(PARTICAO_ID=120260295)
18 Row(PARTICAO_ID=120260294)
19 Row(PARTICAO_ID=120260293)
20 Row(PARTICAO_ID=120260292)
21 Row(PARTICAO_ID=120260291)
22 Row(PARTICAO_ID=120260290)
23 Row(PARTICAO_ID=120260289)
24 Row(PARTICAO_ID=120260288)
25 Row(PARTICAO_ID=120260287)
26 Row(PARTICAO_ID=120260286)
27 Row(PARTICAO_ID=120260285)
28 Row(PARTICAO_ID=120260284)
29 Row(PARTICAO_ID=120260283)
30 Row(PARTICAO_ID=120260282)
31 Row(PARTICAO_ID=120260281)
32 Row(PARTICAO_ID=120260280)
33 Row(PARTICAO_ID=1

In [56]:
for chave, particao in enumerate(particoes):
    particao_id = particao['PARTICAO_ID']
    print(chave,particao_id)
    particao_df = df_total_votos_candidato.filter(df_total_votos_candidato['PARTICAO_ID'] == particao_id).drop('PARTICAO_ID', 'ID_LINHA')
    particao_pandas = particao_df.toPandas()
    nome_arquivo = f'../data/processed/total_votos_candidato/df_total_votos_candidatos_{particao_id}.csv'
    particao_pandas.to_csv(nome_arquivo, header=True, sep='|')

0 120260312


1 120260311
2 120260310
3 120260309


4 120260308


5 120260307


6 120260306
7 120260305


...

8711 61
8712 60
8713 59
8714 58
8715 57
8716 56
8717 55
8718 54
8719 53


8720 52
8721 51
8722 50
8723 49
8724 48
8725 47
8726 46
8727 45


8728 44
8729 43
8730 42
8731 41
8732 40
8733 39
8734 38
8735 37
8736 36


8737 35
8738 34
8739 33
8740 32
8741 31
8742 30
8743 29
8744 28
8745 27
8746 26
8747 25
8748 24
8749 23
8750 22
8751 21
8752 20
8753 19
8754 18


8755 17
8756 16
8757 15
8758 14
8759 13
8760 12
8761 11
8762 10
8763 9
8764 8
8765 7
8766 6
8767 5
8768 4
8769 3
8770 2
8771 1
8772 0


In [ ]:
con = hive.Connection(database='ELEICOES', host='localhost', port=10000)

In [ ]:
cursor = con.cursor()

try:
        

    query = f'''
        LOAD DATA LOCAL INPATH '/home/rodrigo/Documentos/projetos/eleicoes/data/processed/total_votos_candidato.parquet'
        INTO TABLE total_votos_candidato
        '''
    print(query)
        
    cursor.execute(query)
except:
    cursor.close()
    print(f'deu ruim ')
    

cursor.close()


        LOAD DATA LOCAL INPATH '/home/rodrigo/Documentos/projetos/eleicoes/data/processed/total_votos_candidato.parquet'
        INTO TABLE total_votos_candidato
        
deu ruim 
